In [2]:
pip install tensorflow

  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
     ---------------------------------------- 266.3/266.3 MB ? eta 0:00:00
     --------------------------------------- 57.5/57.5 kB 48.7 kB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 kB 136.1 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 345.9 kB/s eta 0:00:00
     ------------------------------------ 895.9/895.9 kB 358.7 kB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 155.7 kB/s eta 0:00:00
     -------------------------------------- 23.2/23.2 MB 198.7 kB/s eta 0:00:00
     ------------------------------------- 126.5/126.5 kB 41.8 kB/s eta 0:00:00
     ------------------------------------ 439.2/439.2 kB 261.4 kB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 115.0 kB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 215.7 kB/s eta 0:00:00
     ------------------------------------ 781.3/781.3 kB 12

In [3]:
from numpy import loadtxt


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
import pandas as pd

In [27]:
train = pd.read_csv('train_clean.csv', )
test = pd.read_csv('test_clean.csv')
df = pd.concat([train, test], axis=0, sort=True)

In [86]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           1309 non-null   float64
 1   Family_Size   1309 non-null   float64
 2   Fare          1309 non-null   float64
 3   Parch         1309 non-null   float64
 4   Pclass        1309 non-null   float64
 5   Sex           1309 non-null   int8   
 6   SibSp         1309 non-null   float64
 7   Survived      891 non-null    float64
 8   Embarked_C    1309 non-null   uint8  
 9   Embarked_Q    1309 non-null   uint8  
 10  Embarked_S    1309 non-null   uint8  
 11  Title_Dr      1309 non-null   uint8  
 12  Title_Master  1309 non-null   uint8  
 13  Title_Miss    1309 non-null   uint8  
 14  Title_Mr      1309 non-null   uint8  
 15  Title_Mrs     1309 non-null   uint8  
 16  Title_Rev     1309 non-null   uint8  
dtypes: float64(7), int8(1), uint8(9)
memory usage: 94.6 KB


,Age,Family_Size,Fare,Parch,Pclass,Sex,SibSp,Survived,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rev
0,-0.574635,0.073352,-0.503176,-0.445,0.841916,1,0.481288,0.0,0,0,1,0,0,0,1,0,0
1,0.641140,0.073352,0.734809,-0.445,-1.546098,0,0.481288,1.0,1,0,0,0,0,0,0,1,0
2,-0.270692,-0.558346,-0.490126,-0.445,0.841916,0,-0.479087,1.0,0,0,1,0,0,1,0,0,0
3,0.413182,0.073352,0.383263,-0.445,-1.546098,0,0.481288,1.0,0,0,1,0,0,0,0,1,0
4,0.413182,-0.558346,-0.487709,-0.445,0.841916,1,-0.479087,0.0,0,0,1,0,0,0,1,0,0


In [87]:
df['Sex'] = df['Sex'].astype('category')
# convert to category codes
df['Sex'] = df['Sex'].cat.codes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           1309 non-null   float64
 1   Family_Size   1309 non-null   float64
 2   Fare          1309 non-null   float64
 3   Parch         1309 non-null   float64
 4   Pclass        1309 non-null   float64
 5   Sex           1309 non-null   int8   
 6   SibSp         1309 non-null   float64
 7   Survived      891 non-null    float64
 8   Embarked_C    1309 non-null   uint8  
 9   Embarked_Q    1309 non-null   uint8  
 10  Embarked_S    1309 non-null   uint8  
 11  Title_Dr      1309 non-null   uint8  
 12  Title_Master  1309 non-null   uint8  
 13  Title_Miss    1309 non-null   uint8  
 14  Title_Mr      1309 non-null   uint8  
 15  Title_Mrs     1309 non-null   uint8  
 16  Title_Rev     1309 non-null   uint8  
dtypes: float64(7), int8(1), uint8(9)
memory usage: 94.6 KB


In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from numpy.random import seed
from tensorflow.random import set_seed 



In [30]:
categorical = ['Embarked', 'Title']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [31]:
df.drop(['Cabin', 'Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

In [34]:
continuous = ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp', 'Family_Size']

scaler = StandardScaler()

for var in continuous:
    df[var] = df[var].astype('float64')
    df[var] = scaler.fit_transform(df[var].values.reshape(-1, 1))

In [ ]:
X_train = df[pd.notnull(df['Survived'])].drop(['Survived'], axis=1)
y_train = df[pd.notnull(df['Survived'])]['Survived']

In [95]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           891 non-null    float64
 1   Family_Size   891 non-null    float64
 2   Fare          891 non-null    float64
 3   Parch         891 non-null    float64
 4   Pclass        891 non-null    float64
 5   Sex           891 non-null    int8   
 6   SibSp         891 non-null    float64
 7   Embarked_C    891 non-null    uint8  
 8   Embarked_Q    891 non-null    uint8  
 9   Embarked_S    891 non-null    uint8  
 10  Title_Dr      891 non-null    uint8  
 11  Title_Master  891 non-null    uint8  
 12  Title_Miss    891 non-null    uint8  
 13  Title_Mr      891 non-null    uint8  
 14  Title_Mrs     891 non-null    uint8  
 15  Title_Rev     891 non-null    uint8  
dtypes: float64(6), int8(1), uint8(9)
memory usage: 57.4 KB


In [76]:
def create_model(lyrs=[8], act='linear', opt='Adam', dr=0.0):
    
    seed(42)
    set_seed(42)
    model = Sequential()
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
  
    for i in range(1,len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
  
    model.add(Dropout(dr))
    

    model.add(Dense(1, activation='sigmoid')) #op layer
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [77]:
model = create_model()
print(model.summary())


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_17 (Dense)            (None, 1)                 9         
                                                                 
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________
None


In [78]:
import numpy as np
from tensorflow.keras.optimizers import Adam

In [93]:
training = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.3, verbose=1)


Epoch 1/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4294 - accuracy: 0.8170 - val_loss: 0.3552 - val_accuracy: 0.8657
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4292 - accuracy: 0.8170 - val_loss: 0.3564 - val_accuracy: 0.8657
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4288 - accuracy: 0.8170 - val_loss: 0.3577 - val_accuracy: 0.8619
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4285 - accuracy: 0.8154 - val_loss: 0.3586 - val_accuracy: 0.8619
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4281 - accuracy: 0.8170 - val_loss: 0.3596 - val_accuracy: 0.8619
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4279 - accuracy: 0.8138 - val_loss: 0.3603 - val_accuracy: 0.8619
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4279 - accuracy: 0.8138 - val_loss: 0.3611 - val_accuracy: 0.8619
Epoch 8/100
2

In [92]:
model.fit(X_train, y_train, epochs=35, batch_size=8)

Epoch 1/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.8260
Epoch 2/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4184 - accuracy: 0.8328
Epoch 3/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.8316
Epoch 4/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4147 - accuracy: 0.8328
Epoch 5/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4142 - accuracy: 0.8316
Epoch 6/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4134 - accuracy: 0.8283
Epoch 7/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4134 - accuracy: 0.8316
Epoch 8/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4131 - accuracy: 0.8316
Epoch 9/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4130 - accuracy: 0.8305
Epoch 10/35
112/112 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.8305

In [94]:
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

28/28 [==============================] - 0s 546us/step - loss: 0.4122 - accuracy: 0.8316
Accuracy: 83.16


In [65]:
# model = Sequential()
# model.add(Dense(8, input_dim=8, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [71]:
# opt=Adam(learning_rate=0.001)
# model.compile(optimizer=opt,loss='binary_crossentropy', metrics=['accuracy'])